# Customer Churn Model Using ANN

Import Libraries 

In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

Data Preprocessing

In [ ]:
data =pd.read_csv('/Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [ ]:
data.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
RowNumber,10000.0,5.000500e+03,2886.895680,1.00,2500.75,5.000500e+03,7.500250e+03,10000.00
CustomerId,10000.0,1.569094e+07,71936.186123,15565701.00,15628528.25,1.569074e+07,1.575323e+07,15815690.00
CreditScore,10000.0,6.505288e+02,96.653299,350.00,584.00,6.520000e+02,7.180000e+02,850.00
Age,10000.0,3.892180e+01,10.487806,18.00,32.00,3.700000e+01,4.400000e+01,92.00
Tenure,10000.0,5.012800e+00,2.892174,0.00,3.00,5.000000e+00,7.000000e+00,10.00
Balance,10000.0,7.648589e+04,62397.405202,0.00,0.00,9.719854e+04,1.276442e+05,250898.09
NumOfProducts,10000.0,1.530200e+00,0.581654,1.00,1.00,1.000000e+00,2.000000e+00,4.00
HasCrCard,10000.0,7.055000e-01,0.455840,0.00,0.00,1.000000e+00,1.000000e+00,1.00
IsActiveMember,10000.0,5.151000e-01,0.499797,0.00,0.00,1.000000e+00,1.000000e+00,1.00
EstimatedSalary,10000.0,1.000902e+05,57510.492818,11.58,51002.11,1.001939e+05,1.493882e+05,199992.48


Encoding categorical Data

In [ ]:
Gender = pd.get_dummies(data['Gender'], prefix='Sex',drop_first=True)
data = pd.concat([data,Gender], axis = 1)
data.drop(columns=['Gender'], axis =1, inplace=True)
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Sex_Male
0,1,15634602,Hargrave,619,France,42,2,0.00,1,1,1,101348.88,1,0
1,2,15647311,Hill,608,Spain,41,1,83807.86,1,0,1,112542.58,0,0
2,3,15619304,Onio,502,France,42,8,159660.80,3,1,0,113931.57,1,0
3,4,15701354,Boni,699,France,39,1,0.00,2,0,0,93826.63,0,0
4,5,15737888,Mitchell,850,Spain,43,2,125510.82,1,1,1,79084.10,0,0


In [ ]:
Geography = pd.get_dummies(data['Geography'], prefix='Country', drop_first=True)
data = pd.concat([data,Geography], axis = 1)
data.drop(columns=['Geography'], axis =1, inplace=True)
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Sex_Male,Country_Germany,Country_Spain
0,1,15634602,Hargrave,619,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,2,15647311,Hill,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1
2,3,15619304,Onio,502,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,4,15701354,Boni,699,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,5,15737888,Mitchell,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1


Splitting the Dataset into Dependent and Independent Variables & Training and Test Set

In [ ]:
X = data[['CreditScore','Country_Germany', 'Country_Spain','Sex_Male','Age','Tenure','Balance','NumOfProducts','HasCrCard','IsActiveMember', 'EstimatedSalary']].values
y = data['Exited'].values

In [ ]:
print(X)
print(y)

[[6.1900000e+02 0.0000000e+00 0.0000000e+00 ... 1.0000000e+00
  1.0000000e+00 1.0134888e+05]
 [6.0800000e+02 0.0000000e+00 1.0000000e+00 ... 0.0000000e+00
  1.0000000e+00 1.1254258e+05]
 [5.0200000e+02 0.0000000e+00 0.0000000e+00 ... 1.0000000e+00
  0.0000000e+00 1.1393157e+05]
 ...
 [7.0900000e+02 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  1.0000000e+00 4.2085580e+04]
 [7.7200000e+02 1.0000000e+00 0.0000000e+00 ... 1.0000000e+00
  0.0000000e+00 9.2888520e+04]
 [7.9200000e+02 0.0000000e+00 0.0000000e+00 ... 1.0000000e+00
  0.0000000e+00 3.8190780e+04]]
[1 0 1 ... 1 1 0]


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Feature Scaling 

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
#y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
#y_test = np.asarray(y_test).astype('float32').reshape((-1,1))

In [ ]:
print(X_train)

[[-1.01460667 -0.5698444   1.74309049 ...  0.64259497 -1.03227043
   1.10643166]
 [-1.01460667  1.75486502 -0.57369368 ...  0.64259497  0.9687384
  -0.74866447]
 [ 0.98560362 -0.5698444  -0.57369368 ...  0.64259497 -1.03227043
   1.48533467]
 ...
 [ 0.98560362 -0.5698444  -0.57369368 ...  0.64259497 -1.03227043
   1.41231994]
 [-1.01460667 -0.5698444   1.74309049 ...  0.64259497  0.9687384
   0.84432121]
 [-1.01460667  1.75486502 -0.57369368 ...  0.64259497 -1.03227043
   0.32472465]]


Buiding an Artificial Neural Network

In [ ]:
ann= tf.keras.models.Sequential()

In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu')) #first layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu')) #second Layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid')) #output layer

In [ ]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
ann.fit(X_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.5390 - accuracy: 0.7960
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4710 - accuracy: 0.7999
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4441 - accuracy: 0.8048
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4328 - accuracy: 0.8066
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4253 - accuracy: 0.8081
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4188 - accuracy: 0.8080
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4132 - accuracy: 0.8116
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4075 - accuracy: 0.8155
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4022 - accuracy: 0.8180
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3975 - accura

Results and Evaluation

In [ ]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print('Confusion Matrix:')
print(confusion_matrix(y_test,y_pred))
print('Classification Report')
print(classification_report(y_test,y_pred))
print('Accuracy Score:', accuracy_score(y_test,y_pred))

Confusion Matrix:
[[1521   74]
 [ 195  210]]
Classification Report
              precision    recall  f1-score   support

           0       0.89      0.95      0.92      1595
           1       0.74      0.52      0.61       405

    accuracy                           0.87      2000
   macro avg       0.81      0.74      0.76      2000
weighted avg       0.86      0.87      0.86      2000

Accuracy Score: 0.8655


Predicting a Single Value

In [ ]:
print(ann.predict(scaler.transform([[600,0,0,1,40,3,60000,2,1,1,50000]]))) #returns probability of leaving 
print(ann.predict(scaler.transform([[600,0,0,1,40,3,60000,2,1,1,50000]])) > 0.5) #adding threshold 0.5 for exit 

[[0.02549165]]
[[False]]
